# Cleaning Data

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
from bs4 import BeautifulSoup


In [9]:
# import raw data

users_df = pd.read_csv('/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/raw-data/raw_users_oct4E.csv')
revs_df = pd.read_csv('/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/raw-data/raw_revs_oct4E.csv')
hostels_df = pd.read_csv('/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/raw-data/raw_hostels_oct4E_ND.csv')

In [10]:
users_df.shape, revs_df.shape,hostels_df.shape

((11498, 5), (23003, 9), (5642, 14))

In [11]:
# look at data

display("users_df.head(2)",users_df.head(2),'revs_df.head(2)',revs_df.head(2),
        'hostels_df.head(2)',hostels_df.head(2))

'users_df.head(2)'

,Unnamed: 0,Unnamed: 0.1,user_id,about,num_revs
0,0,101,1189474,\n ...,2
1,1,102,1595059,\n ...,2


'revs_df.head(2)'

,Unnamed: 0,Unnamed: 0.1,text,user_id,rev_date,score,hostel,location,link
0,0,182,Excellent value!,1189474,30th Jun 2004,7.7,Fortuna Botel,"Budapest, Hungary",http://www.hostelworld.com/hosteldetails.php/F...
1,1,185,"I HAD A GREAT TIME THERE BECAUSE OF THE BEACH,...",1595059,23rd Dec 2004,7.7,Equity Point Sea,"Barcelona, Spain",http://www.hostelworld.com/hosteldetails.php/E...


'hostels_df.head(2)'

,Unnamed: 0,rating,desc,FREE,GENERAL,SERVICES,FOOD_DRINK,ENTERTAINMENT,POLICIES,hostel,location,num_revs,num_awards,rev_highlights
0,0,2.0,[\n Stay with us near to th...,[\n ...,[<p>\n ...,[<p>\n ...,[<p>\n ...,[<p>\n ...,[],Fortuna Botel,"Budapest, Hungary",[],0,[]
1,1,7.5,[\n The Equity Point Sea is...,[\n ...,[<p>\n ...,[<p>\n ...,[<p>\n ...,[<p>\n ...,"[<p>\n Age Restriction<i class=""fa ...",Equity Point Sea,"Barcelona, Spain",[],0,"[<span class=""ratingfactorlabel"">Perfect <b>Lo..."


In [12]:
# defs to remove tags and ugly things

def no_space_n(value):
    return value.replace('\n','').replace('  ','') 
def no_r(value):
    return value.replace('\r','') 
def no_n(value):
    return value.replace('\n','') 
def no_nc(value):
    return value.replace('\n,','').replace(',\n','') 
def add_c(value):
    return value.replace('  ',',')
def del_ec(value):
    return value.replace(',,,','').replace(',,',',')
def del_ec2(value):
    return value.replace(',,',',').replace(',,,',',').replace(',,',',')
def no_sc(value):
    return value.replace('.','').replace(',','').replace('!','').replace('?','').replace('*','').replace('-','').replace('_','').replace("'","").replace('/','').replace(':','').replace(";","")

def soupit(value):
    soup = BeautifulSoup(str(value))
    value = ''.join(soup.findAll(text=True))
    return value

def no_uni(value):
    return value.decode('unicode_escape').encode('ascii','ignore')

def rv_s(value):
    return value[1:]

## Clean Hostels_DF
- Clean FREE, desc, GENERAL, SERVICES, FOOD_DRINK, ENTERTAINMENT, POLICIES
- split location into country and city

In [13]:
# hostels_df.info()
# hostels_df.shape
# hostels_df.isnull().sum()
# hostels_df.num_revs.unique()

In [14]:
# clean hostel df

hostels_df['FREE'] = hostels_df['FREE'].map(no_space_n)
hostels_df['desc'] = hostels_df['desc'].map(no_space_n).map(no_r)
hostels_df['GENERAL'] = hostels_df['GENERAL'].map(soupit).map(add_c).map(del_ec).map(no_nc).map(no_uni)
hostels_df['SERVICES'] = hostels_df['SERVICES'].map(soupit).map(add_c).map(del_ec).map(no_nc) #.map(no_uni)
hostels_df['FOOD_DRINK'] = hostels_df['FOOD_DRINK'].map(soupit).map(add_c).map(del_ec).map(no_nc) #.map(no_uni)
hostels_df['ENTERTAINMENT'] = hostels_df['ENTERTAINMENT'].map(soupit).map(add_c).map(del_ec).map(no_nc).map(no_uni)
hostels_df['POLICIES'] = hostels_df['POLICIES'].map(soupit).map(add_c).map(del_ec2).map(no_nc).map(no_uni)
hostels_df['rev_highlights'] = hostels_df['rev_highlights'].map(soupit)

# split location
hostels_df['country'] = [a.split(",")[1] for a in hostels_df.location]
hostels_df['city'] = [a.split(",")[0] for a in hostels_df.location]
hostels_df.country = hostels_df.country.map(rv_s)

# make hostel desc lowercase and remove special characters
hostels_df.desc = [x.lower() for x in hostels_df.desc]
hostels_df['desc'] = hostels_df['desc'].map(no_sc)

# del unnamed col
del hostels_df['Unnamed: 0']
del hostels_df['num_revs']

In [15]:
# look at cleanded hostels_df

hostels_df.head(2)

,rating,desc,FREE,GENERAL,SERVICES,FOOD_DRINK,ENTERTAINMENT,POLICIES,hostel,location,num_awards,rev_highlights,country,city
0,2.0,[stay with us near to the city center of budap...,"[Linen Included , Free City Maps , Towels Incl...","[Breakfast Not Included, Air Conditioning, Out...","[Airport Transfers, Bicycle Hire, Luggage Stor...","[Restaurant, Meals Available, ]","[Wi-Fi, ]",[],Fortuna Botel,"Budapest, Hungary",0,[],Hungary,Budapest
1,7.5,[the equity point sea is a lively hostel locat...,"[Free Breakfast , Free Internet Access ]","[Wheelchair Friendly, Security Lockers, Common...","[Luggage Storage, 24 Hour Reception, Tours/Tra...","[Restaurant, Bar, ]","[Games Room, ]","[Age Restriction, Credit Cards Accepted, No Cu...",Equity Point Sea,"Barcelona, Spain",0,"[Perfect Location, Brilliant Staff, Good Clean...",Spain,Barcelona


In [213]:
# save cleaned hostels
#hostels_df.to_csv(path_or_buf='/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/clean-data/clean_hostels2_oct4.csv',encoding='utf8')

## Cleaning Users_DF
- clean about column and split into nationality, age, and travel group 
- clean age to have categories 41+, 31-40, 25-30, 18-24, and not specified

In [16]:
# users_df.info()
# users_df.shape
# users_df.isnull().sum()

In [17]:
# clean users about column

users_df.about = users_df.about.map(no_space_n)

In [18]:
# split about into 3 columns 

users_df['nationality'] = [a.split(",")[0] for a in users_df.about]
users_df['group'] = [a.split(",")[1] if len(a.split(",")) == 3 else np.nan for a in users_df.about]
users_df['age'] = [a.split(",")[2] if len(a.split(",")) == 3 else np.nan for a in users_df.about]

# del about and unnamed col

del users_df['about']
del users_df['Unnamed: 0']

In [19]:
# def to clean users age column

def clean_age(value):
    
    if value == 'THIRTYONETOFOURTY':
        value = '31-40'
    if value == 'FOURTYONEPLUS':
        value = '41+'
    if value == 'TWENTYFIVETOTHIRTY':
        value = '25-30'
    if value == 'EIGHTEENTOTWENTYFOUR':
        value = '18-24'
    if value not in  ['41+', '31-40', '25-30','18-24']:
        value = 'not specified'

    return value

# map age cleaning def 
users_df['age'] = users_df['age'].map(clean_age)

In [20]:
# look at cleaned users_df

users_df.head(2)

,Unnamed: 0.1,user_id,num_revs,nationality,group,age
0,101,1189474,2,England,NaN,not specified
1,102,1595059,2,Hong Kong,NaN,not specified


In [21]:
# save cleaned users
#users_df.to_csv(path_or_buf='/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/clean-data/clean_users_oct4.csv',encoding='utf8')

## Cleaning Rev_DF
- split location into country and city
- split date into day, month, and year

In [22]:
# revs_df.info()
# revs_df.shape
# revs_df.isnull().sum()

In [23]:
# split location 

revs_df['country'] = [a.split(",")[1] for a in revs_df.location]
revs_df['city'] = [a.split(",")[0] for a in revs_df.location]
del revs_df['Unnamed: 0.1']

# split rev_date inot dat month and year

revs_df['day'] = [int(d.split(' ')[1].replace('th','').replace('rd','').replace('st','').replace('nd','')) for d in revs_df.rev_date]
revs_df['month'] = [d.split(' ')[2] for d in revs_df.rev_date]
revs_df['year'] = [int(d.split(' ')[3]) for d in revs_df.rev_date]

def month_converter(month):
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    return months.index(month) + 1

revs_df['month'] = revs_df['month'].map(month_converter)

#make rev text all lower case and remove special characters
revs_df.text = [x.lower() for x in revs_df.text]
revs_df.text = revs_df['text'].map(no_sc)

# remove space at front of country
revs_df.country = revs_df.country.map(rv_s)

In [24]:
# def to detect if text is english 

def isEnglish(s):
    try:
        s.decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True
    
# collect index where rev text is not english 

index_nE = []
for i,t in enumerate(revs_df.text):
    if isEnglish(t) == False:
        index_nE.append(i)

In [25]:
revs_en =[i for i in revs_df.index if i not in index_nE]
revs_df_en = revs_df.loc[revs_df.index.isin(revs_en)].reset_index()
del revs_df_en['index']

In [26]:
# look at cleaned revs_df

revs_df_en.head(2)

,Unnamed: 0,text,user_id,rev_date,score,hostel,location,link,country,city,day,month,year
0,0,excellent value,1189474,30th Jun 2004,7.7,Fortuna Botel,"Budapest, Hungary",http://www.hostelworld.com/hosteldetails.php/F...,Hungary,Budapest,30,6,2004
1,1,i had a great time there because of the beach ...,1595059,23rd Dec 2004,7.7,Equity Point Sea,"Barcelona, Spain",http://www.hostelworld.com/hosteldetails.php/E...,Spain,Barcelona,23,12,2004


In [27]:
revs_df_en.shape

(21675, 13)

## Remove users from users_df where users No loger have reviews in revs_df

In [28]:
revs_exist_uid =[u for u in revs_df_en.user_id]
users_df_en = users_df.loc[users_df.user_id.isin(revs_exist_uid)].reset_index()
del users_df_en['index']

In [29]:
users_df_en.shape

(10929, 6)

## Remove hostels from hostels_df where hostels no longer have reviews

In [30]:
revs_exist_hst =[h for h in revs_df_en.hostel]
hostels_df_en = hostels_df.loc[hostels_df.hostel.isin(revs_exist_hst)].reset_index()
del hostels_df_en['index']

In [31]:
hostels_df_en.shape

(5453, 14)

## Make Hostel and country ID's

In [33]:
# create DFs for country and user ids
hostels_df_en['hostel_id'] = [i for i in hostels_df_en.index]

hostel_ids_df = pd.DataFrame(hostels_df_en['hostel'],hostels_df_en['hostel_id'])
hostel_ids_df= hostel_ids_df.reset_index()

country_ids_df = pd.DataFrame(users_df_en.nationality.unique())
country_ids_df=country_ids_df.reset_index()
country_ids_df.columns = ['country_id','country']

In [34]:
# def to add country id to dfs 
def c_id(value):
    for i,c in enumerate(country_ids_df.country):
        if str(c) == str(value):
            return country_ids_df.country_id[i]
        
        
# def to add hostel id to dfs
def h_id(value):
    for i,h in enumerate(hostel_ids_df.hostel):
        if str(h) == str(value):
            return hostel_ids_df.hostel_id[i]

In [35]:
hostels_df_en['country_id'] = hostels_df_en['country'].map(c_id)
users_df_en['country_id'] = users_df_en['nationality'].map(c_id)
revs_df_en['country_id'] = revs_df_en['country'].map(c_id)

In [36]:
revs_df_en['hostel_id'] = revs_df_en['hostel'].map(h_id)

In [55]:
del revs_df_en['Unnamed: 0']
del users_df_en['Unnamed: 0.1']

In [56]:
# Save clean DFs

hostels_df_en.to_csv(path_or_buf='/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/clean-data/clean_hostles_oct7.csv',encoding='utf8')
users_df_en.to_csv(path_or_buf='/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/clean-data/clean_users_oct7.csv',encoding='utf8')
revs_df_en.to_csv(path_or_buf='/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/clean-data/clean_revs_oct7.csv',encoding='utf8')

In [58]:
# save Hostel and Country Ids

country_ids_df.to_csv(path_or_buf='/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/clean-data/country_ids_oct7.csv',encoding='utf8')
hostel_ids_df.to_csv(path_or_buf='/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/clean-data/hostles_ids_oct7.csv',encoding='utf8')